In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark import SparkFiles
sc = SparkContext()
sqlContext = SQLContext(sc)

In [2]:
%%html
<style>
div.output_area pre {
    white-space: pre;
    word-wrap: normal;
}
.container { 
    width:100% !important; 
}
</style>

In [3]:
import pandas as pd

In [4]:
columns = ['deviceID', 'transID', 'transType', 'appid', 'pmcID', 'amount', 'userID', 'osVer', 'deviceModel', 'userIP', 'reqDate', 'longitude', 'latitude', 'campaignID', 'type_transaction']
pdF = pd.read_csv('./tpe_device_score.csv', names=columns, header=None)
pdF.to_csv('./tpe_device_score_with_header.csv', index=False)

In [32]:
from pyspark.sql.functions import lit, udf, col, weekofyear, countDistinct, desc
from pyspark.sql.types import *
from pyspark.sql.functions import lit
from pyspark.sql import Window
import pyspark.sql.functions as f

In [5]:
pdFH = pd.read_csv('./tpe_device_score_with_header.csv')
pdFH.head()

,deviceID,transID,transType,appid,pmcID,amount,userID,osVer,deviceModel,userIP,reqDate,longitude,latitude,campaignID,type_transaction
0,000195863904dc8a,200808000071150,1,454,39,2000000,200119000040347,Android 28 (9),Samsung SM-J730G,125.214.48.174,2020-08-08 08:51:08.856,105.952103,10.223810,0,payment
1,000195863904dc8a,200808000089357,1,454,39,2000000,200119000040347,Android 28 (9),Samsung SM-J730G,125.214.48.174,2020-08-08 09:47:11.279,105.952107,10.223811,0,payment
2,000195863904dc8a,200809000160928,1,454,39,2000000,200119000040347,Android 28 (9),Samsung SM-J730G,125.214.48.174,2020-08-09 14:43:45.359,105.952099,10.223812,0,payment
3,000195863904dc8a,200822000188187,1,454,39,2000000,200119000040347,Android 28 (9),Samsung SM-J730G,125.214.48.174,2020-08-22 14:33:30.566,105.952106,10.223818,0,payment
4,000195863904dc8a,200822000271777,1,454,39,2000000,200119000040347,Android 28 (9),Samsung SM-J730G,125.214.48.174,2020-08-22 20:30:36.476,105.952090,10.223808,0,payment


In [6]:
tpe_df = sqlContext.createDataFrame(pdFH)

In [7]:
tpe_df.show()

+--------------------+---------------+---------+-----+-----+-------+---------------+--------------+----------------+--------------+--------------------+------------------+------------------+----------+----------------+
|            deviceID|        transID|transType|appid|pmcID| amount|         userID|         osVer|     deviceModel|        userIP|             reqDate|         longitude|          latitude|campaignID|type_transaction|
+--------------------+---------------+---------+-----+-----+-------+---------------+--------------+----------------+--------------+--------------------+------------------+------------------+----------+----------------+
|    000195863904dc8a|200808000071150|        1|  454|   39|2000000|200119000040347|Android 28 (9)|Samsung SM-J730G|125.214.48.174|2020-08-08 08:51:...|       105.9521027|        10.2238103|         0|         payment|
|    000195863904dc8a|200808000089357|        1|  454|   39|2000000|200119000040347|Android 28 (9)|Samsung SM-J730G|125.214.

In [8]:
appid_merchant_pd = pd.read_excel('./AppID Merchant.xlsx')
appid_merchant_pd = appid_merchant_pd[['Merchant', 'App ID']]
appid_merchant_pd.head()
appid_merchant_df = sqlContext.createDataFrame(appid_merchant_pd)
appid_merchant_df.show()

+----------------+------+
|        Merchant|App ID|
+----------------+------+
|        App Test|     3|
|        App Test|   742|
|        App Test| 10033|
|        App Test| 10034|
|Business Service|    26|
|Business Service|    48|
|Business Service|   114|
|Business Service|   203|
|Business Service|   204|
|Business Service|   207|
|Business Service|   215|
|Business Service|   230|
|Business Service|   233|
|Business Service|   234|
|Business Service|   235|
|Business Service|   238|
|Business Service|   253|
|Business Service|   259|
|Business Service|   272|
|Business Service|   274|
+----------------+------+
only showing top 20 rows



In [9]:
left_join = tpe_df.join(appid_merchant_df, tpe_df.appid == appid_merchant_df['App ID'])

In [15]:
appid_merchant_df= appid_merchant_df.withColumnRenamed('App ID','appid')

In [16]:
left_join.cache()

DataFrame[deviceID: string, transID: bigint, transType: bigint, appid: bigint, pmcID: bigint, amount: bigint, userID: bigint, osVer: string, deviceModel: string, userIP: string, reqDate: string, longitude: double, latitude: double, campaignID: bigint, type_transaction: string, Merchant: string, App ID: bigint]

In [27]:
joined_df = tpe_df.join(appid_merchant_df,how='left', on='appid')

In [34]:
joined_df.groupBy('deviceID', 'Merchant').count().orderBy('deviceID',f.col('count').desc()).show()

+--------------------+--------------+-----+
|            deviceID|      Merchant|count|
+--------------------+--------------+-----+
|000081FB-CAE0-4DB...|Money Transfer|    1|
|0000B1B7-9075-487...|Money Transfer|    6|
|0000B625-66C9-4B2...|Money Transfer|    6|
|0000B625-66C9-4B2...| Digital Goods|    4|
|00017D00-6F9E-495...|Money Transfer|   12|
|00017D00-6F9E-495...| Digital Goods|    3|
|00017D00-6F9E-495...|       Utility|    1|
|    000195863904dc8a|Money Transfer|   16|
|    000195863904dc8a| Digital Goods|    2|
|0001B388-47BF-4AC...| Digital Goods|    1|
|00020A08-BF1A-473...| Digital Goods|    4|
|00020A08-BF1A-473...|Money Transfer|    4|
|00021A5E-07D6-41F...|       Utility|    1|
|000236E1-DE27-483...|Money Transfer|    1|
|00024F09-CC7E-42D...|Money Transfer|    3|
|00024F09-CC7E-42D...|   Marketplace|    1|
|00028081-26BE-415...| Digital Goods|    1|
|0002D759-1F9A-4A2...| Digital Goods|    5|
|00036630-600D-4F6...|Money Transfer|    1|
|0003D11F-6744-4C5...|       Uti

In [35]:
joined_df.show()

+-----+--------------------+---------------+---------+-----+-------+---------------+------------------+-------------------+---------------+--------------------+---------+--------+----------+----------------+-------------+
|appid|            deviceID|        transID|transType|pmcID| amount|         userID|             osVer|        deviceModel|         userIP|             reqDate|longitude|latitude|campaignID|type_transaction|     Merchant|
+-----+--------------------+---------------+---------+-----+-------+---------------+------------------+-------------------+---------------+--------------------+---------+--------+----------+----------------+-------------+
|   29|3D84B3FB-0F2D-4DF...|200810000099557|        1|   37| 500000|180616000003177|            12.4.1|          iPhone9,4|  27.67.143.125|2020-08-10 09:59:...|      0.0|     0.0|         0|         payment|Digital Goods|
|   29|    a62f48e11bfa500c|200807000154372|        1|   38|1200000|200202000013674|Android 27 (8.1.0)|BlackBerr

In [47]:
joined_df.toPandas().to_csv('./result.csv')

Py4JJavaError: An error occurred while calling o187.collectToPython.
: java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.next(SparkPlan.scala:368)
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.next(SparkPlan.scala:364)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.foreach(SparkPlan.scala:364)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeCollect$1(SparkPlan.scala:386)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeCollect$1$adapted(SparkPlan.scala:385)
	at org.apache.spark.sql.execution.SparkPlan$$Lambda$2904/0x0000000841043040.apply(Unknown Source)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at or